In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   validation_split=0.2)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(120, 120),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=32,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(120, 120),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=32,
        subset='validation')

Found 1162 images belonging to 2 classes.
Found 290 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [7]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50
37/37 [==============================] - 41s 764ms/step - loss: 0.5533 - accuracy: 0.7235 - val_loss: 0.3805 - val_accuracy: 0.8483
Epoch 2/50
37/37 [==============================] - 7s 185ms/step - loss: 0.1681 - accuracy: 0.9417 - val_loss: 0.2895 - val_accuracy: 0.8897
Epoch 3/50
37/37 [==============================] - 7s 176ms/step - loss: 0.1074 - accuracy: 0.9639 - val_loss: 0.2894 - val_accuracy: 0.8759
Epoch 4/50
37/37 [==============================] - 6s 165ms/step - loss: 0.0967 - accuracy: 0.9712 - val_loss: 0.2353 - val_accuracy: 0.9034
Epoch 5/50
37/37 [==============================] - 5s 145ms/step - loss: 0.0745 - accuracy: 0.9732 - val_loss: 0.2312 - val_accuracy: 0.9138
Epoch 6/50
37/37 [==============================] - 7s 182ms/step - loss: 0.0954 - accuracy: 0.9616 - val_loss: 0.1812 - val_accuracy: 0.9345
Epoch 7/50
37/37 [==============================] - 5s 130ms/step - loss: 0.0581 - accuracy: 0.9787 - val_loss: 0.2202 - val_accuracy: 0.9483
Epoch